# Credit Card Fraud Analysis and Detection

---

## Problem Identification

### Problem statement

How can credit card companies detect up to 90% of fraudulent transactions?

### Key data sources

- [Credit Card Fraud Detection Kaggle dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud)

---

## Environment setup

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

sns.set('notebook')

In [2]:
# Read CSV to Dataframe
df = pd.read_csv('../data/raw/creditcard.csv')

In [3]:
# Inspect the data
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [19]:
from sklearn.metrics import confusion_matrix, plot_precision_recall_curve, plot_roc_curve

def metrics_plots(model, X_test, y_test):
    '''Plot Recall Precision AUC'''    
    y_pred = model.predict(X_test)
    fig = plt.figure(figsize=(15,5))

    # Confusion matrix 
    ax = fig.add_subplot(121)
    labels = ['Normal', 'Fraud'] 
    conf_matrix = confusion_matrix(y_test, y_pred) 
    sns.heatmap(conf_matrix, xticklabels=labels, yticklabels=labels, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax); 
    plt.title("Confusion matrix") 
    plt.xlabel('True class') 
    plt.ylabel('Predicted class') 
    
    # AUC graph
    ax = fig.add_subplot(122)
    plot_precision_recall_curve(model, X_test, y_test, ax=ax)
    
    plt.show()

In [20]:
from sklearn.metrics import accuracy_score, auc, f1_score, matthews_corrcoef, precision_recall_curve, precision_score, recall_score

def metrics(model, X_test, y_test):
    '''Print out metrics'''
    # Predict
    y_pred = model.predict(X_test)    
    probabilities = model.predict_proba(X_test)[:,1]

    # Calculate AUC
    precision, recall, _ = precision_recall_curve(y_test, probabilities)
    auc_score = auc(recall, precision)
    
    print(f'Predicted labels: {(np.unique(y_pred))}')
    print(pd.DataFrame(y_pred)[0].value_counts())
    
    print(f'\nAccuracy:  {accuracy_score(y_test, y_pred):.4}')
    print(f'Recall:    {recall_score(y_test, y_pred):.4}')
    print(f'Precision: {precision_score(y_test, y_pred):.4}')
    
    # This style will make it easier to copy and paste for a Markdown table
    print(f'\nAUC    | F1 Score | MCC Score')
    print(f'{auc_score:.4} | {f1_score(y_test, y_pred):.4}   | {matthews_corrcoef(y_test, y_pred):.4}')

In [21]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(model, X_test, y_test, save_path='conf-matrix.png'):
    '''Plot the confusion matrix for a model'''
    y_pred = model.predict(X_test)
    
    labels = ['Normal', 'Fraud'] 
    conf_matrix = confusion_matrix(y_test, y_pred) 

    plt.figure(figsize=(5, 5)) 
    sns.heatmap(conf_matrix, xticklabels=labels, yticklabels=labels, annot=True, fmt="d", cmap="Blues", cbar=False); 
    plt.title("Confusion matrix") 
    plt.xlabel('True class') 
    plt.ylabel('Predicted class')
    
    plt.savefig(save_path, bbox_inches = 'tight')
    plt.show()

In [22]:
from sklearn.metrics import plot_precision_recall_curve

def plot_auc_results(models, X_test, y_test, save_path='prauc-results.png'):
    '''Plot Recall Precision AUC results'''
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    
    for model in models:
        plot_precision_recall_curve(model, X_test, y_test, ax=ax)
    
    plt.savefig(save_path, bbox_inches = 'tight')
    plt.show()

## Hyperparameter Optimization

In [41]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [42]:
# NOTE: this takes >100 minutes to complete
# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 10, 
#                                cv = 3, 
#                                verbose=2, 
#                                random_state=42, 
#                                n_jobs = 2)
# rf_random.fit(X_train, y_train)

best_params = {
    'bootstrap': False,
    'max_depth': 80,
    'max_features': 'auto',
    'min_samples_leaf': 1,
    'min_samples_split': 10,
    'n_estimators': 94,
    'random_state': 42,
    'verbose': 1,
    'n_jobs': -1
}